In [2]:
import re
import heapq

def clean_and_tokenize_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s.]', '', text)
    sentences = text.split('.')
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    return sentences

def calculate_edit_distance(str1, str2):
    len_str1, len_str2 = len(str1), len(str2)
    dp = [[0] * (len_str2 + 1) for _ in range(len_str1 + 1)]

    for i in range(len_str1 + 1):
        dp[i][0] = i
    for j in range(len_str2 + 1):
        dp[0][j] = j

    for i in range(1, len_str1 + 1):
        for j in range(1, len_str2 + 1):
            if str1[i - 1] == str2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = min(dp[i - 1][j] + 1, dp[i][j - 1] + 1, dp[i - 1][j - 1] + 1)

    return dp[len_str1][len_str2]

class AlignmentState:
    def __init__(self, index1, index2, cost, parent=None):
        self.index1 = index1
        self.index2 = index2
        self.cost = cost
        self.parent = parent

    def __lt__(self, other):
        return self.cost < other.cost

def perform_a_star_alignment(doc1, doc2):
    initial_state = AlignmentState(0, 0, 0)
    open_set = []
    heapq.heappush(open_set, (0, initial_state))
    closed_set = set()

    while open_set:
        _, current_state = heapq.heappop(open_set)

        if current_state.index1 == len(doc1) and current_state.index2 == len(doc2):
            return current_state

        closed_set.add((current_state.index1, current_state.index2))

        if current_state.index1 < len(doc1) and current_state.index2 < len(doc2):
            sentence1 = doc1[current_state.index1]
            sentence2 = doc2[current_state.index2]
            new_cost = current_state.cost + calculate_edit_distance(sentence1, sentence2)
            new_state = AlignmentState(current_state.index1 + 1, current_state.index2 + 1, new_cost, current_state)
            if (new_state.index1, new_state.index2) not in closed_set:
                heapq.heappush(open_set, (new_state.cost, new_state))

        if current_state.index1 < len(doc1):
            new_cost = current_state.cost + 1
            new_state = AlignmentState(current_state.index1 + 1, current_state.index2, new_cost, current_state)
            if (new_state.index1, new_state.index2) not in closed_set:
                heapq.heappush(open_set, (new_state.cost, new_state))

        if current_state.index2 < len(doc2):
            new_cost = current_state.cost + 1
            new_state = AlignmentState(current_state.index1, current_state.index2 + 1, new_cost, current_state)
            if (new_state.index1, new_state.index2) not in closed_set:
                heapq.heappush(open_set, (new_state.cost, new_state))

    return None

def backtrack_alignment_path(goal_state, doc1, doc2):
    alignment_result = []
    current = goal_state

    while current:
        idx1, idx2 = current.index1, current.index2
        if idx1 > 0 and idx2 > 0:
            alignment_result.append((doc1[idx1 - 1], doc2[idx2 - 1]))
        elif idx1 > 0:
            alignment_result.append((doc1[idx1 - 1], "-"))
        elif idx2 > 0:
            alignment_result.append(("-", doc2[idx2 - 1]))
        current = current.parent

    alignment_result.reverse()
    return alignment_result

def plagiarism_check(doc1, doc2):
    processed_doc1 = clean_and_tokenize_text(doc1)
    processed_doc2 = clean_and_tokenize_text(doc2)

    goal_state = perform_a_star_alignment(processed_doc1, processed_doc2)

    if goal_state is None:
        print("No alignment found.")
        return []

    alignment = backtrack_alignment_path(goal_state, processed_doc1, processed_doc2)

    detected_plagiarism = []
    for sentence1, sentence2 in alignment:
        if sentence1 != "-" and sentence2 != "-":
            dist = calculate_edit_distance(sentence1, sentence2)
            print(f"Comparing: '{sentence1}' with '{sentence2}', Edit Distance: {dist}")
            if dist < len(sentence1) * 0.5:
                detected_plagiarism.append((sentence1, sentence2))

    return detected_plagiarism

doc1 = "The quick brown fox jumps over the lazy dog. It swiftly runs across the yard. Testing examples are essential for validation."
doc2 = "A fast brown fox leaps over a lazy dog. It quickly moves through the field. Example tests are important for verification."

doc1_identical = doc1
doc2_identical = doc1
print("Test Case 1: Identical Documents")
print(plagiarism_check(doc1_identical, doc2_identical))

print("\nTest Case 2: Slightly Modified Document")
print(plagiarism_check(doc1, doc2))

doc3 = "This document is completely unrelated. There are no similarities here."
print("\nTest Case 3: Completely Different Documents")
print(plagiarism_check(doc1, doc3))

doc4 = "The quick brown fox jumps over the lazy dog. Another random sentence. Testing examples are essential for validation."
print("\nTest Case 4: Partial Overlap")
print(plagiarism_check(doc1, doc4))


Test Case 1: Identical Documents
Comparing: 'the quick brown fox jumps over the lazy dog' with 'the quick brown fox jumps over the lazy dog', Edit Distance: 0
Comparing: 'it swiftly runs across the yard' with 'it swiftly runs across the yard', Edit Distance: 0
Comparing: 'testing examples are essential for validation' with 'testing examples are essential for validation', Edit Distance: 0
[('the quick brown fox jumps over the lazy dog', 'the quick brown fox jumps over the lazy dog'), ('it swiftly runs across the yard', 'it swiftly runs across the yard'), ('testing examples are essential for validation', 'testing examples are essential for validation')]

Test Case 2: Slightly Modified Document
Comparing: 'it swiftly runs across the yard' with 'a fast brown fox leaps over a lazy dog', Edit Distance: 31
Comparing: 'it swiftly runs across the yard' with 'it quickly moves through the field', Edit Distance: 17
Comparing: 'testing examples are essential for validation' with 'it quickly moves t